In [1]:
!pip install docling

  Preparing metadata (setup.py) ... done
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 22.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 22.4 MB/s eta 0:00:0000:0100:01
Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.9 MB/s eta 0

## Document Loading

In [2]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
import pandas as pd

In [3]:
# 1. Configure the pipeline to detect table structure
pipeline_options = PdfPipelineOptions(do_table_structure=True)
# Optional: tweak cell matching or accuracy level
pipeline_options.table_structure_options.do_cell_matching = True
# pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE

In [4]:
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)


In [98]:
# 2. Load/convert the PDF
pdf_path = "/home/ashok/rag-project/docs/Morning-note-and-CA-09-July--25.pdf"  # or URL, stream, etc.
result = converter.convert(pdf_path)

In [99]:
doc = result.document

# Extraction output

In [100]:
!pip install -qU langchain-openai

In [101]:
# 3. Extract and print plain text
print("===== Full Text =====")
print(doc.export_to_markdown())  # or export_to_text()

===== Full Text =====
<!-- image -->

| Indices       | Current Value   |   % 1 D |   % YTD |
|---------------|-----------------|---------|---------|
| Sensex        | 83,713          |     0.3 |     7.1 |
| Nifty         | 25,523          |     0.2 |     7.9 |
| BSE Midcap    | 46,748          |     0   |     0.7 |
| BSE Small cap | 54,559          |    -0.2 |    -1.1 |

## Sectors - Performance (BSE)

| Realty       | 7,626   |   1.1 |   -7.4 |
|--------------|---------|-------|--------|
| Bankex       | 64,037  |   0.7 |   10.9 |
| Power        | 6,917   |   0.7 |   -0.7 |
| Consumer Dur | 60,022  |  -1.7 |   -6.9 |
| Healthcare   | 44,515  |  -0.8 |   -1.7 |
| Telecom      | 3,111   |  -0.5 |    9.2 |

| Nifty Gainers/Losers   | CMP   | %Chg   |
|------------------------|-------|--------|
| KOTAKBANK              | 2,225 | 3.5    |
| ASIANPAINT             | 2,485 | 1.7    |
| NTPC                   | 343   | 1.7    |
| TITAN                  | 3,441 | (5.8)  |
| DRREDDY           

In [102]:
# 4. Extract tables and convert to DataFrames
print("\n===== Extracted Tables =====")
for idx, table in enumerate(doc.tables):
    df: pd.DataFrame = table.export_to_dataframe()
    print(f"\n--- Table {idx} ---")
    print(df)
    # Optionally save:
    # df.to_csv(f"table_{idx}.csv", index=False)


===== Extracted Tables =====

--- Table 0 ---
         Indices Current Value % 1 D % YTD
0         Sensex        83,713   0.3   7.1
1          Nifty        25,523   0.2   7.9
2     BSE Midcap        46,748   0.0   0.7
3  BSE Small cap        54,559  -0.2  -1.1

--- Table 1 ---
         Realty   7,626   1.1  -7.4
0        Bankex  64,037   0.7  10.9
1         Power   6,917   0.7  -0.7
2  Consumer Dur  60,022  -1.7  -6.9
3    Healthcare  44,515  -0.8  -1.7
4       Telecom   3,111  -0.5   9.2

--- Table 2 ---
  Nifty Gainers/Losers    CMP   %Chg
0            KOTAKBANK  2,225    3.5
1           ASIANPAINT  2,485    1.7
2                 NTPC    343    1.7
3                TITAN  3,441  (5.8)
4              DRREDDY  1,284  (2.0)
5                CIPLA  1,488  (1.5)

--- Table 3 ---
  FII Trading activities in Cash       Date    Net    MTD
0                            FII  08-Jul-25     42    364
1                            DII  08-Jul-25  1,331  3,071

--- Table 4 ---
  Global Indices Curr

In [103]:
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv,find_dotenv
import json
import uuid
import re
import os

In [104]:
load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] =  os.getenv("OPENAI_API_KEY")

# Split markdown into table and non-table blocks

In [105]:
markdown_text = doc.export_to_markdown()

In [106]:
def split_markdown_tables(text):
    pattern = r'((?:\|.+\|\n)+)'  # This pattern matches consecutive markdown table lines
    blocks = []
    last_end = 0
    for match in re.finditer(pattern, text):
        start, end = match.span()
        if last_end < start:
            blocks.append(text[last_end:start])  # Non-table block
        blocks.append(match.group())  # Table block
        last_end = end
    if last_end < len(text):
        blocks.append(text[last_end:])  # Any trailing non-table block
    return blocks

In [107]:
blocks = split_markdown_tables(markdown_text)

#create summary of an table

In [108]:
from langchain.prompts import PromptTemplate

def summarize_tables_in_markdown(
    blocks: list,
    model_name: str = "gpt-4o"
) -> str:

    # Initialize the OpenAI GPT-4o model
    llm = ChatOpenAI(model=model_name, temperature=0)

    # Prompt template for table summary
    table_summary_prompt = PromptTemplate(
        input_variables=["table"],
        template=(
            "You are an expert data analyst. Given the following markdown table, "
            "summarize the table without missing key insights,patterns."
            "Extract all information from each record of the table and make into an summary in english"
            "don't miss out the context from the columns, and rows in the tabl;e"
            "These summaries will be embedded and used for retrieval process"
            "Do not repeat the table, only output the summary. Table:\n\n{table}"
        ),
    )

    final_blocks = []

    for block in blocks:
        block_strip = block.strip()
        if block_strip.startswith('|') and block_strip.endswith('|'):
            # Summarize the table block
            prompt = table_summary_prompt.format(table=block)
            summary = llm.invoke(prompt).content.strip()
            final_blocks.append(summary + "\n")
        else:
            final_blocks.append(block)

    final_output = "".join(final_blocks)
    return final_output

In [109]:
final_text_block = summarize_tables_in_markdown(blocks)

In [110]:
print(final_text_block)

<!-- image -->

The table provides a snapshot of the performance of various stock indices. The Sensex index currently stands at 83,713, showing a daily increase of 0.3% and a year-to-date (YTD) growth of 7.1%. The Nifty index is at 25,523, with a 0.2% rise in the last day and a 7.9% increase YTD, indicating a strong performance over the year. The BSE Midcap index is at 46,748, with no change in the daily percentage but a modest YTD growth of 0.7%. In contrast, the BSE Small Cap index is at 54,559, experiencing a slight daily decline of 0.2% and a negative YTD performance of -1.1%, suggesting underperformance compared to the other indices. Overall, while the major indices like Sensex and Nifty show positive trends, the smaller cap indices, particularly the BSE Small Cap, are lagging behind.

## Sectors - Performance (BSE)

The table provides a snapshot of various sectors with their respective indices, percentage changes over a certain period, and their year-to-date performance. The Real

# Semantic chunking and embedding

In [111]:
from langchain.schema import Document

In [112]:
OUTPUT_TEXT_FILENAME = "/content/morning_quote_9th_june.txt"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
CHUNK_SIZE = 700
CHUNK_OVERLAP = 200
EMBEDDINGS_OUTPUT_FILE = "dailymorning_chunks_with_embeddings.json"

In [113]:
def chunk_text(entire_text: list) -> list[Document]:

    print(f"Starting chunking with chunk_size={CHUNK_SIZE}, chunk_overlap={CHUNK_OVERLAP}")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        add_start_index=True,
    )

    # Chunk the text
    chunks = text_splitter.create_documents([entire_text])

    print(f"Text chunking complete. Created {len(chunks)} chunks.")
    return chunks

In [114]:
# Perform semantic chunking
print("\n--- Starting Semantic Chunking ---")
processed_chunks = chunk_text(final_text_block)
if not processed_chunks:
    print("Chunking failed. Exiting.")
    exit()


--- Starting Semantic Chunking ---
Starting chunking with chunk_size=700, chunk_overlap=200
Text chunking complete. Created 145 chunks.


In [119]:
#remove unwanted chunks
final_chunks = processed_chunks[:118]

print("After filtering (only first 100 chunks):")
for i, chunk in enumerate(final_chunks):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n{'-'*40}")

After filtering (only first 100 chunks):
Chunk 1:
<!-- image -->
----------------------------------------
Chunk 2:
The table provides a snapshot of the performance of various stock indices. The Sensex index currently stands at 83,713, showing a daily increase of 0.3% and a year-to-date (YTD) growth of 7.1%. The Nifty index is at 25,523, with a 0.2% rise in the last day and a 7.9% increase YTD, indicating a strong performance over the year. The BSE Midcap index is at 46,748, with no change in the daily percentage but a modest YTD growth of 0.7%. In contrast, the BSE Small Cap index is at 54,559, experiencing a slight daily decline of 0.2% and a negative YTD performance of -1.1%, suggesting underperformance compared to the other indices. Overall, while the major indices like Sensex and Nifty show positive
----------------------------------------
Chunk 3:
a slight daily decline of 0.2% and a negative YTD performance of -1.1%, suggesting underperformance compared to the other indices. Over

In [120]:
def embed_chunks(chunks: list[Document], model_name: str) -> list[Document]:
    print(f"Loading embedding model: {model_name}")
    try:
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
        print("Embedding model loaded successfully.")
    except Exception as e:
        print(f"Error loading embedding model: {e}")
        print("Please ensure 'openai' library is installed: !pip install -qU langchain-openai")
        print("Also check if the model name is correct and accessible.")
        return []

    print(f"Generating embeddings for {len(chunks)} chunks...")
    # Generate embeddings for each chunk's page_content
    # The embeddings object's embed_documents method takes a list of strings
    chunk_texts = [doc.page_content for doc in chunks]
    chunk_embeddings = embeddings.embed_documents(chunk_texts)

    for i, doc in enumerate(chunks):
        doc.metadata["embedding"] = chunk_embeddings[i]
        doc.metadata["chunk_id"] = str(uuid.uuid4())

    print("Embeddings generation complete.")
    return chunks

In [121]:
# Generate embeddings for the chunks
print("\n--- Starting Embedding Generation ---")
chunks_with_embeddings = embed_chunks(final_chunks, EMBEDDING_MODEL_NAME)
if not chunks_with_embeddings:
    print("Embedding generation failed. Exiting.")
    exit()


--- Starting Embedding Generation ---
Loading embedding model: text-embedding-3-small
Embedding model loaded successfully.
Generating embeddings for 118 chunks...
Embeddings generation complete.


In [122]:
# Extract chunks and embeddings
extracted_data = [
    {"page_content": doc.page_content,  "embedding": doc.metadata["embedding"]}
    for doc in chunks_with_embeddings
]

In [123]:

final_chunks

[Document(metadata={'start_index': 0, 'embedding': [-0.03154997527599335, 0.0163394995033741, -0.10805186629295349, -0.024608561769127846, -0.05327324941754341, -0.013015148229897022, 0.05022070184350014, -0.004905508831143379, -0.012189287692308426, -0.07806998491287231, 0.05486224591732025, 0.009523534215986729, 0.053231433033943176, -0.09935419261455536, 0.04008038341999054, 0.0704595148563385, -0.06753241270780563, -0.0436556302011013, 0.04800446704030037, 0.10529202967882156, 0.01536728348582983, -0.04104214534163475, -0.030358225107192993, 0.044868286699056625, 0.04198300093412399, -0.007207987830042839, -0.03037913329899311, 0.0029035801999270916, 0.10269945114850998, -0.030860014259815216, -0.0776936411857605, -0.04938438534736633, 0.02113785594701767, -0.0609673447906971, 0.015639085322618484, -0.005357641261070967, -0.012868792749941349, 0.02108558639883995, -0.05770571902394295, 0.04150211811065674, -0.026887519285082817, -0.046624548733234406, 0.06389444321393967, 0.0129524

# Vector database

In [124]:
import nest_asyncio
nest_asyncio.apply()
import qdrant_client
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, HnswConfig

In [125]:
# --- Configuration for qdrant Connection ---
QDRANT_HOST = "localhost"
QDRANT_PORT = 6333 

COLLECTION_NAME = "document_chunks" 
VECTOR_DIMENSION = 512

EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = None

In [126]:
try:
    print(f"Attempting to connect to Qdrant at {QDRANT_HOST}:{QDRANT_PORT}...")
    client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
    # A simple health check:
    info = client.get_collections()
    print("Qdrant connection successful! Collections:", [c.name for c in info.collections])
except Exception as e:
    print(f"Qdrant connection failed: {e}")
    print("Please ensure your Qdrant Docker container is running (run 'docker ps').")

Attempting to connect to Qdrant at localhost:6333...
Qdrant connection successful! Collections: ['document_chunks_rag', 'document_chat', 'chat_with_docs']


In [127]:
try:
    print(f"Attempting to connect to Qdrant at {QDRANT_HOST}:{QDRANT_PORT}...")
    client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
    # A simple health check:
    info = client.get_collections()
    print("Qdrant connection successful! Collections:", [c.name for c in info.collections])
except Exception as e:
    print(f"Qdrant connection failed: {e}")
    print("Please ensure your Qdrant Docker container is running (run 'docker ps').")


from qdrant_client import QdrantClient
from qdrant_client.http import models

QDRANT_HOST = "localhost"
QDRANT_PORT = 6333
COLLECTION_NAME = "document_chunks_rag"
VECTOR_DIMENSION = 512  # Set to your embedding size

client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

# Drop existing collection if it exists
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)
    print(f"Dropped existing collection '{COLLECTION_NAME}'")

# Create HNSW index configuration
hnsw_config = models.HnswConfigDiff(
    m=16,
    ef_construct=200,
    full_scan_threshold=10000
)

# Create the collection
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=VECTOR_DIMENSION,
        distance=models.Distance.COSINE
    ),
    hnsw_config=hnsw_config,
    on_disk_payload=False
)

print(f"✅ Created Qdrant collection '{COLLECTION_NAME}' with HNSW index (COSINE, m=16, ef=200)")

# Create a list of PointStructs for upsert
points = [
    PointStruct(
        id=i,  # Ensure unique IDs; you can also omit for auto-generated IDs
        vector=record["embedding"],
        payload={"page_content": record["page_content"]}
    )
    for i, record in enumerate(extracted_data, start=1)
]

# Upsert points into the collection
client.upsert(
    collection_name="document_chunks_rag",
    points=points
)

print(f"✅ Inserted {len(points)} records into Qdrant.")

Attempting to connect to Qdrant at localhost:6333...
Qdrant connection successful! Collections: ['document_chunks_rag', 'document_chat', 'chat_with_docs']
Dropped existing collection 'document_chunks_rag'
✅ Created Qdrant collection 'document_chunks_rag' with HNSW index (COSINE, m=16, ef=200)
✅ Inserted 118 records into Qdrant.


In [128]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

QDRANT_HOST = "localhost"
QDRANT_PORT = 6333
COLLECTION_NAME = "document_chunks_rag"
VECTOR_DIMENSION = 512  # Set to your embedding size

client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

# Drop existing collection if it exists
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)
    print(f"Dropped existing collection '{COLLECTION_NAME}'")

# Create HNSW index configuration
hnsw_config = models.HnswConfigDiff(
    m=16,
    ef_construct=200,
    full_scan_threshold=10000
)

# Create the collection
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=VECTOR_DIMENSION,
        distance=models.Distance.COSINE
    ),
    hnsw_config=hnsw_config,
    on_disk_payload=False
)

print(f"✅ Created Qdrant collection '{COLLECTION_NAME}' with HNSW index (COSINE, m=16, ef=200)")

Dropped existing collection 'document_chunks_rag'
✅ Created Qdrant collection 'document_chunks_rag' with HNSW index (COSINE, m=16, ef=200)


In [129]:
from qdrant_client.http.models import PointStruct

In [130]:
# Create a list of PointStructs for upsert
points = [
    PointStruct(
        id=i,  # Ensure unique IDs; you can also omit for auto-generated IDs
        vector=record["embedding"],
        payload={"page_content": record["page_content"]}
    )
    for i, record in enumerate(extracted_data, start=1)
]

# Upsert points into the collection
client.upsert(
    collection_name="document_chunks_rag",
    points=points
)

print(f"✅ Inserted {len(points)} records into Qdrant.")

✅ Inserted 118 records into Qdrant.


# qdrant RETRIEVAL PIPELINE


In [131]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)

query = "Which are all the company has highest upside above 30 percentage for the investment picks under the buy recommendation ?"
query_vec = embeddings.embed_query(query)

In [132]:
results = client.search(collection_name=COLLECTION_NAME, query_vector=query_vec, limit= 10)

for hit in results:
    print(f"ID: {hit.id}, Score: {hit.score:.3f}\n→ {hit.payload['page_content']}\n")

ID: 107, Score: 0.728
→ 48. **Shriram Finance Ltd** shows a potential increase of 17.7%, with a CMP of 921 and a target of 790.

49. **Signatureglobal (India) Ltd** has a target price of 1,470, indicating an 18.5% upside.

Overall, the table highlights a range of companies across various sectors with promising growth potential, as indicated by their respective target prices and percentage upsides.

<!-- image -->

## Investment Picks

The table provides investment recommendations for eight companies, all of which have been given a "BUY" recommendation. Each company is listed with its current market price (CMP), a target price, and the expected percentage upside.

ID: 96, Score: 0.690
→ <!-- image -->

## Investment Picks

The table provides a comprehensive analysis of various companies with a "BUY" recommendation, indicating potential investment opportunities. Each company is evaluated based on its Current Market Price (CMP), Target Price, and the expected percentage upside.

1. **Aart

/tmp/ipykernel_5207/1900161487.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(collection_name=COLLECTION_NAME, query_vector=query_vec, limit= 10)


# Prompt Template and Generate output through LLM

In [133]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(
    model="gpt-4o",      # Load GPT‑4o (Omni)   
)

In [134]:
def retrieve_relevant_chunks_qdrant(query_text: str, embeddings, client, top_k: int = 3):
    # 1. Embed the query
    print(f"Embedding query: {query_text!r}")
    q_vec = embeddings.embed_query(query_text)
    
    # 2. Perform search in Qdrant
    hits = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=q_vec,
        limit=top_k
    )
    
    chunks = [
        hit.payload.get("page_content", "")
        for hit in hits
    ]
    print(f"Retrieved {len(chunks)} chunks.")
    return chunks

In [135]:
def generate_llm_response_gpt4o(query: str, llm_model, context_chunks: list[str]) -> str:

    PROMPT = PromptTemplate.from_template(
    """You are a helpful assistant. Answer the user's question using only the given context below.
    create a complete answer only from the given context. You should not create your own answer which is outside the context."

    Context:
    {context}

    Question:
    {question}

    Answer:"""
    )

    
    context = "\n\n".join(context_chunks) if context_chunks else "No context available."
    message = [
        {"role": "user", "content": PROMPT.format(context=context, question=query)}
    ]
    resp = llm_model.invoke(message)
    return resp.content.strip()

In [136]:
user_query = "Can you give me overall result preview of an chemical industries in Q1FY26?Give me a top picks?"

retrieved_chunks = retrieve_relevant_chunks_qdrant(user_query, embeddings ,client, top_k=5)

# Generate answer with context
answer = generate_llm_response_gpt4o(user_query, llm,retrieved_chunks)
print("Answer:", answer)

Embedding query: 'Can you give me overall result preview of an chemical industries in Q1FY26?Give me a top picks?'


/tmp/ipykernel_5207/651712837.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


Retrieved 5 chunks.
Answer: In Q1FY26, the chemical industries are expected to benefit from stronger demand from key end markets and the resulting operational efficiencies. Most specialty chemical companies under coverage are anticipated to show steady year-over-year improvement, primarily driven by volume growth on a favourable (low) base. However, quarter-over-quarter performance is likely to be mixed due to geopolitical tensions and tariff-related uncertainties. Pricing remains subdued owing to ongoing macroeconomic challenges. Our top picks in the chemical sector are Camlin Fine Science Ltd, Navin Fluorine International Ltd, PI Industries Ltd, Dhanuka Agritech Ltd, and Mold-Tek Packaging Ltd.


In [138]:
user_query = "Which are all the company has highest upside above 30 percentage for the investment picks under the buy recommendation ?"

retrieved_chunks = retrieve_relevant_chunks_qdrant(user_query, embeddings ,client, top_k=5)

# Generate answer with context
answer = generate_llm_response_gpt4o(user_query, llm,retrieved_chunks)
print("Answer:", answer)

Embedding query: 'Which are all the company has highest upside above 30 percentage for the investment picks under the buy recommendation ?'


/tmp/ipykernel_5207/651712837.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


Retrieved 5 chunks.
Answer: The companies with the highest upside above 30% under the buy recommendation are **Pitti Engineering Ltd** with a 40.1% upside and **NLC India Ltd** offering a 33.2% upside.


# MMR Re-ranking 

In [139]:
import numpy as np

In [140]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)

In [141]:
def get_top_k_chunks(query, top_k=20):
    # Embed query
    query_embedding = embeddings.embed_query(query)
    # Qdrant search
    search_result = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=query_embedding,
        limit=top_k,
        with_payload=True,   # Make sure chunk content is in payload
        with_vectors=True    # Get embeddings for MMR
    )
    # Parse to list of dicts: [{'content': ..., 'embedding': ..., 'score': ...}, ...]
    results = []
    for hit in search_result:
        results.append({
            'page_content': hit.payload['page_content'],
            'embedding': hit.vector,
            'score': hit.score    # higher score = more similar in Qdrant
        })
    return results, np.array(query_embedding)

In [142]:
# Step 2: MMR Re-ranking
def mmr(query_embedding, docs, lambda_mult=0.7, k=5):
    doc_embeddings = np.array([doc["embedding"] for doc in docs])
    # Cosine similarity: higher is more similar
    doc_scores = np.dot(doc_embeddings, query_embedding) / (
        np.linalg.norm(doc_embeddings, axis=1) * np.linalg.norm(query_embedding) + 1e-8
    )

    # Similarity between documents
    doc_doc_sim = np.dot(doc_embeddings, doc_embeddings.T)
    doc_doc_sim /= (np.linalg.norm(doc_embeddings, axis=1, keepdims=True) @ np.linalg.norm(doc_embeddings, axis=1, keepdims=True).T + 1e-8)

    selected, unselected = [], list(range(len(docs)))
    for _ in range(min(k, len(docs))):
        mmr_scores = []
        for idx in unselected:
            relevance = doc_scores[idx]
            if selected:
                diversity = max([doc_doc_sim[idx, sel_idx] for sel_idx in selected])
            else:
                diversity = 0
            mmr_score = lambda_mult * relevance - (1 - lambda_mult) * diversity
            mmr_scores.append((mmr_score, idx))
        _, selected_idx = max(mmr_scores, key=lambda x: x[0])
        selected.append(selected_idx)
        unselected.remove(selected_idx)
    return [docs[i] for i in selected]

In [143]:
def generate_llm_response_gpt4o(query: str, llm_model, context_chunks: list[str]) -> str:

    PROMPT = PromptTemplate.from_template(
    """You are a helpful assistant. Answer the user's question using only the given context below.
    create a complete answer only from the given context. You should not create your own answer which is outside the context."

    Context:
    {context}

    Question:
    {question}

    Answer:"""
    )

    
    context = "\n\n".join(context_chunks) if context_chunks else "No context available."
    message = [
        {"role": "user", "content": PROMPT.format(context=context, question=query)}
    ]
    resp = llm_model.invoke(message)
    return resp.content.strip()

In [144]:
# === Full Pipeline Function ===
def rag_pipeline(user_query, top_k=20, mmr_k=5):
    # 1. Retrieve top-k candidates from Qdrant
    docs, query_emb = get_top_k_chunks(user_query, top_k=top_k)
    if not docs:
        print("No relevant chunks found.")
        return ""
    # 2. Re-rank with MMR
    reranked_chunks = mmr(query_emb, docs, lambda_mult=0.7, k=mmr_k)

    # 3. Extract page_content for LLM
    context_chunks = [doc['page_content'] for doc in reranked_chunks]

    # 3. Pass context and query to LLM
    answer = generate_llm_response_gpt4o(user_query, llm,context_chunks)
    
    return answer

In [145]:
query = "Can you give me an overall summary of an IT services result preview in Q1FY26?"
answer = rag_pipeline(query, top_k=15, mmr_k=5)

print("\n--- LLM Final Answer ---\n", answer)

/tmp/ipykernel_5207/1991967692.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(



--- LLM Final Answer ---
 The IT services sector in Q1FY26 is anticipated to show moderate growth, with expected revenue growth in the range of -1% to 2% QoQ in US dollar terms and 0.5% to 2% in rupee terms, aided by cross-currency tailwinds. This growth comes amid weaker discretionary spending and macroeconomic uncertainties, including trade issues like Trump tariffs and potential trade wars. Despite these challenges, the sector benefits from a steady deal pipeline. Margin expansions are expected due to cost optimization efforts, sluggish recruitment, and delayed wage hikes. Indian IT Services clients, particularly in the US and Europe, are focusing on cost optimizations, resulting in increased cost take-out deals and vendor consolidation. The BFSI, Hi-tech, and Healthcare Services industries are likely to exhibit some recovery. Specifically, Tech Mahindra is predicted to report a 0.6% QoQ revenue growth, driven by challenges in its Comviva, manufacturing, and retail businesses, thou

In [146]:
query = "Which are all the company has highest upside above 30 percentage for the investment pick under the buy recommendation ?"
answer = rag_pipeline(query, top_k=15, mmr_k=5)

print("\n--- LLM Final Answer ---\n", answer)

/tmp/ipykernel_5207/1991967692.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(



--- LLM Final Answer ---
 The company with the highest upside above 30% under the buy recommendation is **Varun Beverages Ltd**, which has a potential upside of 41.8%.


# RAG Evaluation

In [88]:
# !pip install ragas

In [87]:
# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context

# generator = TestsetGenerator.from_langchain(
#     generator_llm=llm,
#     critic_llm=llm,
#     embeddings=embeddings
# )

In [86]:
# from langchain_community.document_loaders import DirectoryLoader

In [85]:
# distribution = {simple: 0.5, reasoning: 0.25, multi_context: 0.25}
# testset = generator.generate_with_langchain_docs(final_chunks,
#                                                  test_size=10,
#                                                  distributions=distribution,
#                                                  raise_exceptions=False)